# InPars

Author: Monique Monteiro (moniquelouise@gmail.com)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
main_dir = "/content/gdrive/MyDrive/Unicamp-aula-9"

## Libraries installation

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached openai-0.27.6-py3-none-any.whl (71 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached multidict-6.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached frozenlist-1.3.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (149 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.9 MB/s eta 0:00:00
  Using cached yarl-1.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (268 kB)


In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached jsonlines-3.1.0-py3-none-any.whl (8.6 kB)


In [13]:
!pip install pyserini -q
!pip install faiss-cpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 42.7 MB/s eta 0:00:00


## Pyserini indexing

In [19]:
from pyserini.search.lucene import LuceneSearcher
import numpy as np
import json

The code below is inspired by Manoel Veríssimo dos Santos Neto (https://github.com/verissimomanoel/P_IA368DD_2023S1/blob/main/Exercicio8/generate_dataset.py) and Júlia Tessler (https://colab.research.google.com/drive/1DvFvghq3rpDfj0qh1rdpF_Tr0RXa3yV0?usp=sharing#scrollTo=wIPv9dBzeuG5).

At the moment, Pyserini command line version seem unstable, so programmatic access is necessary (and more simple).

In [17]:
def generate_random_numbers(max = 5, k = 1000):
  random_list = []
  while len(random_list) < max:
    n = np.random.randint(0, k)

    # Prevent duplicated index
    if n not in random_list:
      random_list.append(n)

  return random_list

def search_with_bm25(query, max = 5, k = 1000):
  searcher = LuceneSearcher.from_prebuilt_index('beir-v1.0.0-trec-covid.flat')
  hits = searcher.search(query, k)
  random_list = generate_random_numbers(max = max, k = k)
  random_ids = []

  for index in random_list:
    jsondoc = json.loads(hits[index].raw)
    random_ids.append(jsondoc["_id"])

  return random_ids

In [20]:
test = search_with_bm25('covid')
test

['gvvc48eh', 'q8d7rvnj', 'lfdeowsl', '5guhj6y3', '6i0bmnee']

## Passages download

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz

--2023-05-01 17:21:28--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
Resolving huggingface.co (huggingface.co)... 18.155.68.44, 18.155.68.116, 18.155.68.121, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/e9e97686e3138eaff989f67c04cd32e8f8f4c0d4857187e3f180275b23e24e85?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27corpus.jsonl.gz%3B+filename%3D%22corpus.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1683220694&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvZTllOTc2ODZlMzEzOGVhZmY5ODlmNjdjMDRjZDMyZThmOGY0YzBkNDg1NzE4N2UzZjE4MDI3NWIyM2UyNGU4NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPS

In [ ]:
!mv corpus.jsonl.gz {main_dir}/trec-covid

In [ ]:
!gunzip {main_dir}/trec-covid/corpus.jsonl.gz

## Queries vs passages download

It'll be useful to help to generate reasonable queries which will be furthe improved in terms of commplexity.

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv

--2023-05-01 18:44:58--  https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
Resolving huggingface.co (huggingface.co)... 18.155.68.121, 18.155.68.116, 18.155.68.44, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980831 (958K) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>] 957.84K  --.-KB/s    in 0.04s   

2023-05-01 18:44:59 (26.7 MB/s) - ‘test.tsv’ saved [980831/980831]



In [ ]:
!mv test.tsv {main_dir}/trec-covid

## Queries download

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz

--2023-05-01 19:02:43--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz
Resolving huggingface.co (huggingface.co)... 18.155.68.121, 18.155.68.116, 18.155.68.44, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.121|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/9eadcc2cdf140addc9dae83648bb2c6611f5e4b66eaed7475fa5a0ca48eda371?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27queries.jsonl.gz%3B+filename%3D%22queries.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1683226963&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvOWVhZGNjMmNkZjE0MGFkZGM5ZGFlODM2NDhiYjJjNjYxMWY1ZTRiNjZlYWVkNzQ3NWZhNWEwY2E0OGVkYTM3MT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW

In [ ]:
!mv queries.jsonl.gz {main_dir}/trec-covid

In [ ]:
!gunzip {main_dir}/trec-covid/queries.jsonl.gz

In [ ]:
!head {main_dir}/trec-covid/queries.jsonl

{"_id": "1", "text": "what is the origin of COVID-19", "metadata": {"query": "coronavirus origin", "narrative": "seeking range of information about the SARS-CoV-2 virus's origin, including its evolution, animal source, and first transmission into humans"}}
{"_id": "2", "text": "how does the coronavirus respond to changes in the weather", "metadata": {"query": "coronavirus response to weather changes", "narrative": "seeking range of information about the SARS-CoV-2 virus viability in different weather/climate conditions as well as information related to transmission of the virus in different climate conditions"}}
{"_id": "3", "text": "will SARS-CoV2 infected people develop immunity? Is cross protection possible?", "metadata": {"query": "coronavirus immunity", "narrative": "seeking studies of immunity developed due to infection with SARS-CoV2 or cross protection gained due to infection with other coronavirus types"}}
{"_id": "4", "text": "what causes death from Covid-19?", "metadata": {"

In [ ]:
query_ids = []
query_texts = []

with jsonlines.open(f"{main_dir}/trec-covid/queries.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    query_ids.append(id)
    text = item["text"]
    query_texts.append(text)

## Few-shot learning

In [ ]:
import jsonlines

passage_ids = []
passage_texts = []

with jsonlines.open(f"{main_dir}/trec-covid/corpus.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    passage_ids.append(id)
    text = item["title"] + ' ' + item["text"]
    passage_texts.append(text)

In [ ]:
n_examples = 5
min_doc_length = 300 #I use the same limit for choosing the prompt documents, but I don't know if it's necessary.  I may be harmful to the maximum input size supported by GPT 3.5
n_documents = 1000

In [ ]:
!head {main_dir}/trec-covid/test.tsv

query-id	corpus-id	score
1	005b2j4b	2
1	00fmeepz	1
1	g7dhmyyo	2
1	0194oljo	1
1	021q9884	1
1	02f0opkr	1
1	047xpt2c	0
1	04ftw7k9	0
1	pl9ht0d0	0


In [ ]:
import pandas as pd

df = pd.read_csv(f"{main_dir}/trec-covid/test.tsv", sep='\t')
max_score = df["score"].max()
df.shape

(66336, 3)

In [ ]:
df.head()

,query-id,corpus-id,score
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1


In [ ]:
df = df[df["score"] == max_score]

In [ ]:
df.shape

(14217, 3)

In [ ]:
import random

random.seed(15)
df_selected = None
texts = []

#Tries to sample 3 examples with maximum relevance score until we get 3 samples 
#with at least 300 characters.
while(True):
  random_indices = random.sample(range(df.shape[0]), n_examples)
  df_selected = df.iloc[random_indices]
  corpus_ids = df_selected["corpus-id"]
  for id in corpus_ids:
    text = passage_texts[passage_ids.index(id)]
    if len(text) < min_doc_length:
      continue
    texts.append(text)
  break

df_selected

,query-id,corpus-id,score
18344,12,szsb1oan,2
970,1,ltar1aa5,2
41968,29,gwm36n75,2
57889,41,e76foj38,2
3180,3,7n6pva1l,2


In [ ]:
query_texts[11]

'what are best practices in hospitals and at home in maintaining quarantine?'

In [ ]:
passage_texts[passage_ids.index("szsb1oan")]

'The deadly coronaviruses: The 2003 SARS pandemic and the 2020 novel coronavirus epidemic in China Abstract The 2019-nCoV is officially called SARS-CoV-2 and the disease is named COVID-19. This viral epidemic in China has led to the deaths of over 1800 people, mostly elderly or those with an underlying chronic disease or immunosuppressed state. This is the third serious Coronavirus outbreak in less than 20 years, following SARS in 2002–2003 and MERS in 2012. While human strains of Coronavirus are associated with about 15% of cases of the common cold, the SARS-CoV-2 may present with varying degrees of severity, from flu-like symptoms to death. It is currently believed that this deadly Coronavirus strain originated from wild animals at the Huanan market in Wuhan, a city in Hubei province. Bats, snakes and pangolins have been cited as potential carriers based on the sequence homology of CoV isolated from these animals and the viral nucleic acids of the virus isolated from SARS-CoV-2 infec

In [ ]:
query_texts[0]

'what is the origin of COVID-19'

In [ ]:
passage_texts[passage_ids.index("ltar1aa5")]

'Phylogenomic proximity and comparative proteomic analysis of SARS-CoV-2 The coronavirus disease (COVID-19) belongs to the family Severe Acute Respiratory Syndrome (SARS-CoV). It can be more severe for some persons and can lead to pneumonia or breathing difficulties resulting in the death of immune-compromised patients. We performed a phylogenomic and phylogeographic tree from the collected datasets. Phylogenomic analysis or sequence-based phylogeny showed an evolutionary relationship between the geographical strains. The phylogenomic tree grouped into two major clades consists of various isolates of SARS-CoV-2 and Bat SARS-like coronavirus, Bat coronavirus, and Pangolin coronavirus. The phylogenetic neighbor of newly sequenced Indian strains (Accession: MT012098.1, MT050493.1) was revealed to identify the variations between the nCoV-19 strains. The results showed keen evidence that SARS-CoV-2 has evolved from Bat SARS-like coronavirus. The evolutionary history and comparative proteomi

In [ ]:
query_texts[28]

'which SARS-CoV-2 proteins-human proteins interactions indicate potential for drug targets. Are there approved drugs that can be repurposed based on this information?'

This document (gwm36n75) is generating a strong bias to chloroquine and hydroxychloroquine, so it'll be removed.

In [ ]:
passage_texts[passage_ids.index("gwm36n75")]

'Docking study of Chloroquine and Hydroxychloroquine interaction with SARS-CoV-2 spike glycoprotein-An in silico insight into the comparative efficacy of repurposing antiviral drugs. Recent outbreak of novel Coronavirus disease (Covid-19) pandemic around the world is associated with severe acute respiratory syndrome. The death toll associated with the pandemic is increasing day by day. SARS-CoV-2 is an enveloped virus and there are ample evidences that its Spike protein (S-protein) is mainly associated with pathogenesis as it is surface-exposed and mediates entry into host cells by binding to angiotensin-converting enzyme-2 (ACE-2) receptor. Therefore, it is the main target of neutralizing antibodies upon infection and there is always a quest to inhibit the S-protein which in turn may help in controlling diseases caused by SARS-CoV-2 in humans. The role of Chloroquine and Hydroxychloroquine as potential treatments for Covid-19 is still under debate globally because of some side effects

In [ ]:
query_texts[40]

'What are the impacts of COVID-19 among African-Americans that differ from the rest of the U.S. population?'

In [ ]:
passage_texts[passage_ids.index("e76foj38")]

'Hospitalization and Mortality among Black Patients and White Patients with Covid-19 BACKGROUND: Many reports on coronavirus disease 2019 (Covid-19) have highlighted age- and sex-related differences in health outcomes. More information is needed about racial and ethnic differences in outcomes from Covid-19. METHODS: In this retrospective cohort study, we analyzed data from patients seen within an integrated-delivery health system (Ochsner Health) in Louisiana between March 1 and April 11, 2020, who tested positive for severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2, the virus that causes Covid-19) on qualitative polymerase-chain-reaction assay. The Ochsner Health population is 31% black non-Hispanic and 65% white non-Hispanic. The primary outcomes were hospitalization and in-hospital death. RESULTS: A total of 3626 patients tested positive, of whom 145 were excluded (84 had missing data on race or ethnic group, 9 were Hispanic, and 52 were Asian or of another race or ethnic

In [ ]:
new_queries = ["how important are quarantime measures", 
               "what are COVID-19 biological affiliations", 
               "how does COVID-19 impact african-americans when compared to the rest of the USA population"]

In [ ]:
texts = [
    passage_texts[passage_ids.index("szsb1oan")], 
    passage_texts[passage_ids.index("ltar1aa5")], 
    passage_texts[passage_ids.index("e76foj38")]
    ]
ids = ["szsb1oan", "ltar1aa5", "e76foj38"]

In [ ]:

guidelines_prompt = "Generate one relevant query whose answer is contained in the document.  Your query can start with what, which, how or when.  "
guidelines_prompt += "Here are some examples:"
for i in range(3):
  guidelines_prompt += f"\nDocument: {texts[i]}\n"
  guidelines_prompt += f"Relevant query: {new_queries[i]}"

documents = []
random_indices = random.sample(range(len(passage_texts)), n_documents * 2)
documents = [(passage_ids[i], passage_texts[i]) for i in random_indices 
             if len(passage_texts[i]) >= min_doc_length]
print(len(documents))
documents = documents[:1000]


1470


In [ ]:
key = input("Key:")

In [ ]:

import openai

openai.api_key = key

def openai_chat_completion_response(final_prompt):
  response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "You are generating a new dataset for information retrieval"},
                    {"role": "user", "content": final_prompt}
                ]
            )

  return response

In [ ]:
responses = []

In [ ]:
print(guidelines_prompt)

Generate one relevant query whose answer is contained in the document.  Your query can start with what, which, how or when.  Here are some examples:
Document: The deadly coronaviruses: The 2003 SARS pandemic and the 2020 novel coronavirus epidemic in China Abstract The 2019-nCoV is officially called SARS-CoV-2 and the disease is named COVID-19. This viral epidemic in China has led to the deaths of over 1800 people, mostly elderly or those with an underlying chronic disease or immunosuppressed state. This is the third serious Coronavirus outbreak in less than 20 years, following SARS in 2002–2003 and MERS in 2012. While human strains of Coronavirus are associated with about 15% of cases of the common cold, the SARS-CoV-2 may present with varying degrees of severity, from flu-like symptoms to death. It is currently believed that this deadly Coronavirus strain originated from wild animals at the Huanan market in Wuhan, a city in Hubei province. Bats, snakes and pangolins have been cited a

In [ ]:
import time

requests_per_min = 3
requests = 0

for document_id, document_text in documents:
  final_prompt = guidelines_prompt
  final_prompt += f"Document: {document_text}\n"
  final_prompt += f"Relevant query: "
  response = openai_chat_completion_response(final_prompt[:4097])
  print(response)
  #print(response["choices"][0]["message"]["content"])
  triplet = (document_id, document_text, response["choices"][0]["message"]["content"])
  responses.append(triplet)

  if(len(responses) % requests_per_min) == 0:
    time.sleep(60)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "What are the racial and ethnic differences in outcomes from Covid-19?",
        "role": "assistant"
      }
    }
  ],
  "created": 1682977432,
  "id": "chatcmpl-7BVbEnKpuUBO0cdOKdeGLqY96Nj0r",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 14,
    "prompt_tokens": 867,
    "total_tokens": 881
  }
}
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "What are the racial and ethnic differences in outcomes from COVID-19?",
        "role": "assistant"
      }
    }
  ],
  "created": 1682977433,
  "id": "chatcmpl-7BVbFRtI8e7UkZjW7XLgQW2FjsEzV",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 14,
    "prompt_tokens": 867,
    "total_tokens": 881
  }
}
{
  "choices": [
    {
      "finish_reason": "stop",
      "index

KeyboardInterrupt: ignored

In [ ]:
len(responses)

3

In [ ]:
responses

[('m1cmkkw3',
  'Rapid identification of novel antigens of Salmonella Enteritidis by microarray-based immunoscreening We report on an approach to rapidly screen thousands of Salmonella Enteritidis proteins with the goal of identifying novel immunodominant proteins. We used a microarray-based system that warrants high throughput and easy handling. Seven immunogenic candidates were selected after screening. Comparative analyses by ELISA and microarrays manifested their immunodominant character. The large repetitive protein (SEN4030) that plays a role as a putative adhesin in initial cell surface interaction and is highly specific to Salmonella is considered to be the most suitable protein for a diagnostic approach. The results further demonstrate that the strategy applied herein is convenient for specifically identifying immunogenic proteins of pathogenic microorganisms. Consequently, it enables a sound assessment of promising candidates for diagnostic applications and vaccine developmen

As we can see from the previous snippets, in a few-shot setup, the model is generating multiple times almost the same query (about racial and ethnic differences). It reflects only the theme in the last example.  The same occurred when the last example mentioned chloroquine.

So, let's test zero-shot.

## Zero-shot learning

In [ ]:
guidelines_prompt = "Generate one relevant query whose answer is contained in the following document.  Your query can start with what, which, how or when.  "

In [ ]:
responses = []

In [ ]:
import time

requests_per_min = 3
requests = 0

for document_id, document_text in documents:
  final_prompt = guidelines_prompt
  final_prompt += f"Document: {document_text}\n"
  final_prompt += f"Relevant query: "
  response = openai_chat_completion_response(final_prompt[:4097])
  #print(response["choices"][0]["message"]["content"])
  triplet = (document_id, document_text, response["choices"][0]["message"]["content"])
  responses.append(triplet)
  print(len(responses))
  #time.sleep(1)

  #if(len(responses) % requests_per_min) == 0:
   # time.sleep(20)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
responses[:10]

[('m1cmkkw3',
  'Rapid identification of novel antigens of Salmonella Enteritidis by microarray-based immunoscreening We report on an approach to rapidly screen thousands of Salmonella Enteritidis proteins with the goal of identifying novel immunodominant proteins. We used a microarray-based system that warrants high throughput and easy handling. Seven immunogenic candidates were selected after screening. Comparative analyses by ELISA and microarrays manifested their immunodominant character. The large repetitive protein (SEN4030) that plays a role as a putative adhesin in initial cell surface interaction and is highly specific to Salmonella is considered to be the most suitable protein for a diagnostic approach. The results further demonstrate that the strategy applied herein is convenient for specifically identifying immunogenic proteins of pathogenic microorganisms. Consequently, it enables a sound assessment of promising candidates for diagnostic applications and vaccine developmen

In [ ]:
len(responses)

1000

In [ ]:
import json

with open(f"{main_dir}/trec-covid/monique_monteiro_1000_queries.jsonl", "w") as f:
  for doc_id, _, query in responses:
    json.dump({"query":query, "positive_doc_id":doc_id, "negative_doc_ids":[]}, f)
    f.write("\n")

In [ ]:
!head {main_dir}/trec-covid/monique_monteiro_1000_queries.jsonl

{"query": "What is the most suitable protein for a diagnostic approach for Salmonella Enteritidis and why?", "positive_doc_id": "m1cmkkw3", "negative_doc_ids": []}
{"query": "What is Cryptosporidium parvum and why is it a major cause of disease in both humans and animals?", "positive_doc_id": "ukbl0svm", "negative_doc_ids": []}
{"query": "What is the role of the renin-angiotensin-aldosterone system in the context of SARS-CoV-2 infection?", "positive_doc_id": "12o4zey2", "negative_doc_ids": []}
{"query": "What are the functions of individual endolysosomal proteases in cellular processes such as autophagy and lipoprotein particle degradation?", "positive_doc_id": "eqv6a7tj", "negative_doc_ids": []}
{"query": "What is the prevalence of olfactory dysfunction in SARS-CoV-2 patients?", "positive_doc_id": "66gu5af1", "negative_doc_ids": []}
{"query": "What are the main differences between cultures of sarcoid origin and controls in terms of cytokine concentration?", "positive_doc_id": "10j46r1

In [ ]:
import pickle

with open(f"{main_dir}/responses.pickle", "wb") as f:
  pickle.dump(responses, f)

In [ ]:
with open(f"{main_dir}/documents.pickle", "wb") as f:
  pickle.dump(documents, f)

In [22]:
import pickle

with open(f"{main_dir}/responses.pickle", "rb") as f:
  responses = pickle.load(f)

## Searching with BM25 the original index to find negative examples

In [23]:
from tqdm import tqdm

negative_samples = []

for _, _, query in tqdm(responses):
  negative_doc_ids = search_with_bm25(query)
  negative_samples.append(negative_doc_ids)


100%|██████████| 1000/1000 [02:59<00:00,  5.56it/s]


In [24]:
negative_samples[0:10]

[['0o3mryu1', 'qpq7i1ya', 'j5mkparg', 'auoo0dm5', 'dqfvrerw'],
 ['k3u2nvpe', 'gc11fyms', 'xoq9qblv', '20o4ufa3', '3huo5nf0'],
 ['6gd6nwpu', 'dt4t2wos', '8zwfkken', 'sv7xpi4f', '6pf73z08'],
 ['gmrty2uu', 'uzn214j6', '032utjfh', 'efet3ozc', '0muwl6oc'],
 ['o6atz33c', 'nm2bq717', '9hvhru6o', '381esemd', 'exvvms1j'],
 ['gnzau8uj', 'mmafv2b5', 'dckuhrlf', '3tkx57g1', 'm38jah1z'],
 ['9v4nlo8o', 'tw057okg', 'qgklt8wa', 'kkga96h9', '1a0fj900'],
 ['uhjp541j', 'w3e6ul3v', 'rs6hsy8u', 'j7novgyb', 'slbiz2vw'],
 ['r6kl7vb0', '6l7fwylu', 'avfvpqs5', 'owsoyvyz', 'mgrxo21j'],
 ['x580ktas', 'p1cuif8t', 'ykl0xmta', '69x6jkp6', 'tlg5mnvk']]

In [25]:
import json

with open(f"{main_dir}/trec-covid/monique_monteiro_1000_queries.jsonl", "w") as f:
  for i, (doc_id, _, query) in enumerate(responses):
    json.dump({"query":query, "positive_doc_id":doc_id, "negative_doc_ids":negative_samples[i]}, f)
    f.write("\n")

In [26]:
!head {main_dir}/trec-covid/monique_monteiro_1000_queries.jsonl

{"query": "What is the most suitable protein for a diagnostic approach for Salmonella Enteritidis and why?", "positive_doc_id": "m1cmkkw3", "negative_doc_ids": ["0o3mryu1", "qpq7i1ya", "j5mkparg", "auoo0dm5", "dqfvrerw"]}
{"query": "What is Cryptosporidium parvum and why is it a major cause of disease in both humans and animals?", "positive_doc_id": "ukbl0svm", "negative_doc_ids": ["k3u2nvpe", "gc11fyms", "xoq9qblv", "20o4ufa3", "3huo5nf0"]}
{"query": "What is the role of the renin-angiotensin-aldosterone system in the context of SARS-CoV-2 infection?", "positive_doc_id": "12o4zey2", "negative_doc_ids": ["6gd6nwpu", "dt4t2wos", "8zwfkken", "sv7xpi4f", "6pf73z08"]}
{"query": "What are the functions of individual endolysosomal proteases in cellular processes such as autophagy and lipoprotein particle degradation?", "positive_doc_id": "eqv6a7tj", "negative_doc_ids": ["gmrty2uu", "uzn214j6", "032utjfh", "efet3ozc", "0muwl6oc"]}
{"query": "What is the prevalence of olfactory dysfunction in 